**This requires > the standard 2GB of memory in docker. It worked with 3.5GB, but less may be possible.**

In [1]:
import pandas as pd
from ebmdatalab import bq
from lib.outliers import *

In [2]:
with open("../data/static_outlier_sql/chem_per_para.sql") as sql:
    query = sql.read()
#chem_per_para = bq.cached_read(query, csv_path='data/chem_per_para.zip')

## reload specifying data type currently required
## due to https://github.com/ebmdatalab/datalab-pandas/issues/26
chem_per_para = pd.read_csv('../data/chem_per_para.zip',dtype={'subpara': str})
chem_per_para.head()

,practice,pcn,ccg,chemical,subpara,numerator
0,C83009,U58704,04D,1001040C0,1001040,712
1,H81069,U67032,10C,0601021M0,0601021,406
2,M83096,U91937,05G,0307000J0,0307000,302
3,A83030,U01989,00J,0202020D0,0202020,148
4,N81117,U85986,02F,0906026M0,0906026,235


In [3]:
## WHAT TO DO WHERE DENOMINATOR == 0?

In [4]:
loop_over_everything(chem_per_para, ['practice','pcn','ccg',])